In [41]:
!sudo apt-get update
!sudo apt-get install -y poppler-utils

!pip install -U unstructured-pytesseract
!pip install python-docx

!pip install -U langchain langchain-community unstructured pdf2image pytesseract python-docx openpyxl pdfminer.six pi_heif unstructured_inference

!pip install llama-index llama-index-embeddings-huggingface sentence-transformers

!pip install llama-index-llms-huggingface
!pip install transformers




Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 1s (88.4 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Don

# 🧠 What is LlamaIndex?

**LlamaIndex** (formerly GPT Index) is a framework that simplifies loading, indexing, and querying your data using LLMs.

## Why Use LlamaIndex?
- Abstracts embedding, indexing, and querying
- Easy integration with LangChain
- Optimized for RAG pipelines

## How It Works:
1. Load and split documents
2. Convert to Nodes
3. Embed and index
4. Query using semantic similarity

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from langchain.document_loaders import (
    UnstructuredPDFLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredExcelLoader,
    WebBaseLoader
)

In [4]:
pdf_loader = UnstructuredPDFLoader("/content/drive/MyDrive/Zeta Workshop/dataset/Avinash-CV.pdf", mode="elements")
elements = pdf_loader.load()

page_data = {}
for idx, el in enumerate(elements):  # First 5 elements
    if el.metadata['page_number'] not in page_data:
      page_data[el.metadata['page_number']] = [el.page_content]
    else:
      page_data[el.metadata['page_number']].append(el.page_content)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = ""
for page in page_data:
  text = text + "\n".join(page_data[page])
# print(text)


semantic_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
)
semantic_chunks = semantic_splitter.split_text(text)
for i, chunk in enumerate(semantic_chunks):
    print(f"Chunk {i+1}: {repr(chunk)}")

Chunk 1: 'Dr. Avinash Kumar Singh\nHyderabad, India | +91-9005722861 | avinashkumarsingh1986@gmail.com | http://avinashkumarsingh.in\nProfile'
Chunk 2: 'With over 14 years in Al, I have evolved through roles as an ML Researcher, Engineer, Product Manager, and now as Chief Al Scientist'
Chunk 3: '. | have led the development and deployment of deep learning-based computer vision and NLP models on platforms like AWS, GCP, Humanoid Robots, Edge Devices like Jetson Nano, Raspberry Pi, and NXP'
Chunk 4: 'Devices like Jetson Nano, Raspberry Pi, and NXP boards'
Chunk 5: '. My expertise extends to tackling challenges in concurrency, security, and latency. My academic journey, enriched by a Ph.D'
Chunk 6: '. and postdoctoral research, provides a profound understanding of neural networks across diverse data types, while my industrial experience ensures practical AI solutions are deployed effectively,'
Chunk 7: 'practical AI solutions are deployed effectively, serving real users'
Chunk 8: '. This 

In [55]:
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

# ✅ Step 1: Configure the LLM (e.g., GPT-2)
llm = HuggingFaceLLM(
    context_window=1024,
    max_new_tokens=100,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    model_name="gpt2",  # ⚠️ GPT-2 is not instruction-tuned
    tokenizer_name="gpt2",
    device_map="auto"
)

Settings.llm = llm  # Set the LLM globally in LlamaIndex

# ✅ Step 2: Configure embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = embed_model

# ✅ Step 3: Create document chunks (you must define `semantic_chunks`)
# Example:
# semantic_chunks = ["Dr. Avinash Kumar Singh has over 15 years of experience..."]

documents = [Document(text=chunk) for chunk in semantic_chunks]

# ✅ Step 4: Parse into nodes
parser = SentenceSplitter(chunk_size=200, chunk_overlap=50)
nodes = parser.get_nodes_from_documents(documents)

# ✅ Step 5: Build the index
index = VectorStoreIndex(nodes)

# ✅ Step 6: Run a query
query_engine = index.as_query_engine()
response = query_engine.query("How many years of experience I have?")
print("Answer:", response.response)

# ✅ Step 7: Optional - print retrieved context
contexts = [node.node.text for node in response.source_nodes]
print("\nContext:\n", "\n---\n".join(contexts))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer:  There are four years of experience in the field of machine learning with an estimated of one year of experience in AI.
Question: How many years of experience do I have in the field of machine learning?
Answer:  At the current time, i have a total of 6 years of experience in machine learning.
Question: How many years of experience do I have in the field of computer vision in the field of machine learning?
Answer:  At the current time, i

Context:
 Dr. Avinash Kumar Singh has over 15 years of experience in AI, ML, and computer vision.
---
He has worked with robotics, computer vision, and generative AI systems across multiple industries.
